In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Data Processing
target = np.array([0, 2, 0.5])
target = torch.tensor(target).float()

## Load from whole jit model

In [10]:
model = torch.jit.load("CPP_example_model_latest.pt")
p = model(target)
print(p)
f = p.size()
f

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0321, 0.0000, 0.0000, 1.3981,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1988, 0.0000, 4.8071, 0.0000,
        0.7352, 0.0000, 0.0000, 0.0000, 0.4394, 0.5431, 0.0000, 0.0000, 0.0000,
        0.0000, 0.3561, 1.0561, 1.1057, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0636, 1.6730, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2131,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 4.1785, 0.0000, 0.0000, 0.0000, 3.6502,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1615,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

torch.Size([180])

## Load from saved state_dict()

In [63]:
# Load model and Sim

# Generate simulation
sim = d.PySimSeq('test2.sim', 60)
yseq = sim.compute(sim.p)
f = sim.f(sim.y, sim.ydot, sim.yddot, sim.p)
df = sim.df_dp(sim.y, sim.ydot, sim.yddot, sim.p)

class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        return x

#Load model
model_eval = ActiveLearn(input_size, output_size)
model_eval.load_state_dict(torch.load('Trained_Model_300420_300s_100e_onlyZpos.pt'));
model_eval.eval() 

Model(
  (bnorm): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=100, out_features=180, bias=True)
  )
)

In [64]:
with torch.no_grad():
    p = model_eval(target)

In [65]:
# Store p traj
sim.save_ptraj('p_traj.p', p)

In [76]:
ysim = sim.compute(p)

print(f'\ny_0: {ysim[0:3]}') 
print(f'\ny_end: {ysim[59*3:]}') 
print(f'\nDiff: {(abs(ysim[59*3:]-np.array(target)))}') 
print(f'\nError: {sum(abs(ysim[59*3:]-np.array(target)))**2}') 


y_0: [-8.34951326e-05  2.02299677e+00  1.69111837e-03]

y_end: [-0.00816879  2.02603096  0.89845623]

Diff: [0.00816879 0.02603096 0.00154374]

Error: 0.0012775972791816523
